# 🔮 AGE_REQ_DATE Prediction Pipeline - Lateness Forecasting

**Goal:** Generate AGE_REQ_DATE (lateness) predictions for **open deliveries** (not yet shipped).

**Use Case:** Enable operations team to:
- Predict which orders will be late vs customer requested date
- Proactively contact customers about at-risk deliveries
- Prioritize strategic accounts with predicted late deliveries
- Identify patterns in lateness by carrier, plant, customer

**What is AGE_REQ_DATE?**
- **AGE_REQ_DATE** = GI Date - Req. Date Header (Customer Requested Delivery Date)
- **Positive values** = Late (shipped after customer requested date)
- **Negative values** = Early (shipped before customer requested date)
- **Zero** = On-time (shipped on customer requested date)

**Example:**
- Customer requests delivery by Nov 15
- Predicted AGE_REQ_DATE = +3 days
- Predicted ship date = Nov 18 (3 days late)

**Workflow:**
1. Load trained regression model from MLflow
2. Get **open deliveries** using DAX (deliveries without GI Date)
3. Generate AGE_REQ_DATE predictions
4. Calculate predicted ship date: Req. Date Header + predicted AGE_REQ_DATE
5. Categorize lateness and flag at-risk orders
6. Save predictions to Lakehouse table: `delivery_lateness_predictions`
7. View summary statistics and at-risk metrics

### 📦 1. Import Libraries & Configuration

In [ ]:
# ==============================================================================
# IMPORTS & CONFIGURATION
# ==============================================================================

import sempy.fabric as fabric
import pandas as pd
import numpy as np
import mlflow
from datetime import datetime, timedelta

# Configuration
DATASET = "DLV Aging Columns & Measures"  # UPDATE to match your semantic model name
MODEL_NAME = "ship_date_predictor"  # Trained on AGE_REQ_DATE
TARGET_COLUMN = "AGE_REQ_DATE"

print("✅ Configuration loaded")
print(f"   Semantic Model: {DATASET}")
print(f"   Model: {MODEL_NAME}")
print(f"   Target: {TARGET_COLUMN} (lateness vs customer requested date)")

### 🤖 2. Load Trained Model from MLflow

**What's happening:** Load the AGE_REQ_DATE regression model you trained in Notebook 02. This model learned patterns from historical closed deliveries to predict how many days late/early future orders will be.

**Key Details:**
- Model name: `ship_date_predictor`
- Trained on: AGE_REQ_DATE (GI Date - Req. Date Header)
- Performance: MAE ~0.63 days (very accurate!)
- Registry: MLflow automatically tracks versions

In [ ]:
# ==============================================================================
# LOAD MODEL
# ==============================================================================

print("Loading trained model from MLflow...")

model_uri = f"models:/{MODEL_NAME}/latest"
model = mlflow.sklearn.load_model(model_uri)

print(f"✅ Model loaded: {MODEL_NAME}")
print(f"   Type: {type(model).__name__}")
print(f"   URI: {model_uri}")

### 📥 3. Load Open Deliveries from Semantic Model

**What's happening:** Query your Power BI semantic model for all deliveries that haven't shipped yet (blank GI Date). These are the orders we need to predict lateness for.

**Filters Applied:**
- ✅ GI Date is blank (not shipped yet)
- ✅ Delivery Created On exists (valid order)
- ✅ Req. Date Header exists (customer requested delivery date available)

**Why this matters:** We can only predict lateness if we know when the customer wants the order. Without Req. Date Header, we can't calculate AGE_REQ_DATE.

In [ ]:
# ==============================================================================
# LOAD OPEN DELIVERIES
# ==============================================================================
# WHY: Get all deliveries that haven't shipped yet (GI Date is blank)
#      These are the orders we need to predict lateness for
# ==============================================================================

print("Loading open deliveries from semantic model...")

ws = fabric.get_workspace_id()

dax_query = """
EVALUATE
FILTER(
    Aging,
    ISBLANK(Aging[GI Date]) &&
    NOT(ISBLANK(Aging[Delivery Created On])) &&
    NOT(ISBLANK(Aging[Req. Date Header]))
)
"""

df_open = fabric.evaluate_dax(dataset=DATASET, dax_string=dax_query, workspace=ws)

# Clean column names
df_open.columns = [col.split('[')[-1].replace(']', '') if '[' in col else col for col in df_open.columns]

print(f"✅ Loaded {len(df_open):,} open deliveries")
print(f"   Columns: {len(df_open.columns)}")
print(f"\n📊 Sample data:")
df_open.head()

### 🔧 4. Prepare Features for Scoring

**CRITICAL:** Features must exactly match what the model was trained on!

**The 7 Key Features:**
1. Channel (e.g., Retail, Wholesale, Direct)
2. Delivery Priority (e.g., High, Normal, Low)
3. EWM Shipping Condition
4. Shipping Point (which DC/warehouse)
5. Sold To Name 1 (customer name - most important feature!)
6. Standard Or Custom (product type)
7. Product Category

**Data Transformation:**
- Categorical variables → numeric codes (model can't read text)
- Missing values → filled with "Unknown" or median
- Same encoding method used in training (consistency is key!)

In [ ]:
# ==============================================================================
# FEATURE PREPARATION
# ==============================================================================
# CRITICAL: Must match the exact features used during training!
# ==============================================================================

# Features used during training (UPDATE to match your training notebook)
feature_cols = [
    "Channel",
    "Delivery Priority",
    "EWM Shipping Condition",
    "Shipping Point",
    "Sold To Name 1",
    "Standard Or Custom",
    "Product Category"
]

# Filter to available features
available_features = [f for f in feature_cols if f in df_open.columns]

print(f"=== Feature Matching ===")
print(f"Expected features: {len(feature_cols)}")
print(f"Available features: {len(available_features)}")

if len(available_features) < len(feature_cols):
    missing = [f for f in feature_cols if f not in df_open.columns]
    print(f"⚠️ Missing features: {missing}")

# Extract features
X_open = df_open[available_features].copy()

# Encode categorical variables (same as training)
categorical_cols = X_open.select_dtypes(include=['object', 'string']).columns.tolist()
for col in categorical_cols:
    X_open[col] = X_open[col].fillna('Unknown')
    X_open[col] = X_open[col].astype('category').cat.codes

# Handle numeric NaNs
numeric_cols = X_open.select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    if X_open[col].isnull().sum() > 0:
        X_open[col] = X_open[col].fillna(X_open[col].median())

print(f"\n✅ Prepared {len(X_open):,} records for scoring")
print(f"   Features: {len(available_features)} columns")

### 🔮 5. Generate AGE_REQ_DATE Predictions

**What the model predicts:** How many days late/early each delivery will be compared to the customer's requested delivery date.

**Interpreting Predictions:**
- **+5.2 days** = Will ship 5.2 days AFTER customer requested date (LATE)
- **-1.3 days** = Will ship 1.3 days BEFORE customer requested date (EARLY)
- **0 days** = Will ship exactly on customer requested date (ON-TIME)

**Accuracy:** Model is typically accurate within ±0.63 days based on test performance. Most predictions will be very close to actual outcomes.

In [ ]:
# ==============================================================================
# GENERATE PREDICTIONS
# ==============================================================================
# WHAT: Model predicts AGE_REQ_DATE (days late/early vs customer request)
#       Positive = Late, Negative = Early, 0 = On-time
# ==============================================================================

print("\n" + "="*60)
print("GENERATING AGE_REQ_DATE PREDICTIONS")
print("="*60)

# Make predictions
predictions = model.predict(X_open)

# Add predictions to dataframe
df_open['predicted_age_req_date'] = predictions

print(f"✅ Generated {len(predictions):,} predictions")
print(f"\n📊 Prediction Statistics:")
print(f"   Mean:   {predictions.mean():.2f} days")
print(f"   Median: {np.median(predictions):.2f} days")
print(f"   Min:    {predictions.min():.2f} days (early)")
print(f"   Max:    {predictions.max():.2f} days (late)")
print(f"   Std:    {predictions.std():.2f} days")
print(f"\n📈 Distribution:")
print(f"   Predicted Early (<0):    {(predictions < 0).sum():,} ({(predictions < 0).sum()/len(predictions)*100:.1f}%)")
print(f"   Predicted On-Time (0):   {(predictions == 0).sum():,} ({(predictions == 0).sum()/len(predictions)*100:.1f}%)")
print(f"   Predicted Late (>0):     {(predictions > 0).sum():,} ({(predictions > 0).sum()/len(predictions)*100:.1f}%)")
print(f"   Predicted Very Late (>5): {(predictions > 5).sum():,} ({(predictions > 5).sum()/len(predictions)*100:.1f}%)")

### 📅 6. Calculate Predicted Ship Date

Calculate when each delivery will actually ship by adding the predicted lateness to the customer's requested date.

In [ ]:
# ==============================================================================
# CALCULATE PREDICTED SHIP DATE
# ==============================================================================
# predicted_ship_date = Req. Date Header + predicted_age_req_date
# ==============================================================================

df_open['predicted_ship_date'] = (
    pd.to_datetime(df_open['Req. Date Header']) + 
    pd.to_timedelta(df_open['predicted_age_req_date'], unit='d')
)

print("✅ Calculated predicted ship dates")

### 🏷️ 7. Categorize Predictions & Flag At-Risk Orders

**Business-Friendly Categories:**
- 🟢 **Early** - Shipped before customer requested date
- 🟡 **On-Time** - Within 3 days of customer requested date
- 🔴 **Late** - More than 3 days after customer requested date

**At-Risk Flags:**
- `at_risk` = TRUE if predicted >3 days late (action required)

**Use Case:** Operations team can filter to at-risk orders each morning and proactively contact customers before issues escalate.

In [ ]:
# ==============================================================================
# CATEGORIZE LATENESS
# ==============================================================================

def categorize_lateness(days):
    """Categorize predicted AGE_REQ_DATE into simple buckets"""
    if pd.isna(days):
        return "Unknown"
    elif days < 0:
        return "Early"
    elif days <= 3:
        return "On-Time"
    else:
        return "Late"

df_open['lateness_category'] = df_open['predicted_age_req_date'].apply(categorize_lateness)

# Flag at-risk deliveries (predicted >3 days late)
df_open['at_risk'] = df_open['predicted_age_req_date'] > 3

print("✅ Categorized predictions and flagged at-risk orders")
print(f"\n📊 Distribution by Lateness Category:")
print(df_open['lateness_category'].value_counts().sort_index())
print(f"\n🚨 At-Risk Deliveries (>3 days late): {df_open['at_risk'].sum():,}")

### 💾 8. Save Predictions to Lakehouse

**Output Table:** `delivery_lateness_predictions`

**What's included:**
- **Identifiers:** Delivery Number, Customer name
- **Order Details:** Priority, Product category
- **Dates:** Customer requested date
- **Predictions:** AGE_REQ_DATE, ship date, lateness category, risk flags
- **Metadata:** Prediction timestamp, model name

**Next Steps:**
1. Add this table to your Power BI semantic model
2. Create relationship: `Aging[Delivery Number]` → `delivery_lateness_predictions[Delivery Number]`
3. Build dashboards using prediction columns
4. Schedule this notebook to run daily at 6 AM for fresh predictions

In [ ]:
# ==============================================================================
# SAVE TO LAKEHOUSE
# ==============================================================================

print("\n=== Saving Predictions to Lakehouse ===")

# Select essential columns for Power BI
output_cols = [
    # Identifiers
    'Delivery Number',
    'Sold To Name 1',
    
    # Order details
    'Delivery Priority',
    'Product Category',
    
    # Dates
    'Req. Date Header',
    
    # Predictions
    'predicted_age_req_date',
    'predicted_ship_date',
    'lateness_category',
    'at_risk'
]

# Filter to columns that exist
available_output_cols = [c for c in output_cols if c in df_open.columns]
predictions_df = df_open[available_output_cols].copy()

# Add metadata
predictions_df['prediction_timestamp'] = datetime.now()
predictions_df['model_name'] = MODEL_NAME

# Save to Lakehouse table
table_name = "delivery_lateness_predictions"
spark_df = spark.createDataFrame(predictions_df)
spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)

print(f"✅ Saved {len(predictions_df):,} predictions to table: {table_name}")
print(f"✅ Columns saved: {len(available_output_cols) + 2}")
print(f"\n🚨 At-Risk Summary:")
print(f"   Total At-Risk Deliveries: {df_open['at_risk'].sum():,} (predicted >3 days late)")

---

## ✅ Predictions Complete!

The `delivery_lateness_predictions` table is now available in your Lakehouse and ready for Power BI consumption.

**Key Outputs:**
- `predicted_age_req_date`: Days late/early vs customer requested delivery date
- `predicted_ship_date`: Forecasted ship date (Req. Date + predicted lateness)
- `lateness_category`: Simple grouping (Early, On-Time, Late)
- `at_risk`: Flag for deliveries predicted >3 days late

**Next Steps:**
1. **Add to Power BI**: Import `delivery_lateness_predictions` table into your semantic model
2. **Create Relationship**: `Aging[Delivery Number]` → `delivery_lateness_predictions[Delivery Number]`
3. **Build Dashboards**: Create at-risk delivery views, customer lists, carrier performance reports
4. **Automate**: Schedule this notebook to run daily at 6 AM
5. **Monitor**: Track prediction accuracy by comparing to actuals weekly
6. **Take Action**: Use at-risk data for daily operations standup meetings and proactive customer outreach